Pentru a rula următoarele notebook-uri, dacă nu ai făcut-o deja, trebuie să setezi cheia openai în fișierul .env ca `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

În continuare, vom încărca Indexul de Încapsulare într-un Dataframe Pandas. Indexul de Încapsulare este stocat într-un fișier JSON numit `embedding_index_3m.json`. Indexul de Încapsulare conține Încapsulările pentru fiecare dintre transcrierile YouTube până la sfârșitul lunii octombrie 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

În continuare, vom crea o funcție numită `get_videos` care va căuta în Embedding Index pentru interogarea dată. Funcția va returna primele 5 videoclipuri care sunt cele mai asemănătoare cu interogarea. Funcția funcționează astfel:

1. Mai întâi, se creează o copie a Embedding Index.
2. Apoi, se calculează Embedding-ul pentru interogare folosind OpenAI Embedding API.
3. După aceea, se creează o coloană nouă în Embedding Index numită `similarity`. Coloana `similarity` conține similaritatea cosinus dintre Embedding-ul interogării și Embedding-ul fiecărui segment video.
4. În continuare, Embedding Index este filtrat după coloana `similarity`. Embedding Index va include doar videoclipurile care au o similaritate cosinus mai mare sau egală cu 0.75.
5. La final, Embedding Index este sortat după coloana `similarity` și sunt returnate primele 5 videoclipuri.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Această funcție este foarte simplă, doar afișează rezultatele interogării de căutare.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Mai întâi, Indexul de Embedding este încărcat într-un Dataframe Pandas.
2. Apoi, utilizatorul este invitat să introducă o interogare.
3. După aceea, funcția `get_videos` este apelată pentru a căuta interogarea în Indexul de Embedding.
4. În final, funcția `display_results` este apelată pentru a afișa rezultatele utilizatorului.
5. Utilizatorul este apoi invitat să introducă o altă interogare. Acest proces continuă până când utilizatorul introduce `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ro.png)

Vei fi invitat să introduci o interogare. Scrie o interogare și apasă enter. Aplicația va returna o listă de videoclipuri relevante pentru interogarea ta. De asemenea, aplicația va oferi un link către momentul din videoclip unde se află răspunsul la întrebare.

Iată câteva exemple de interogări pe care le poți încerca:

- Ce este Azure Machine Learning?
- Cum funcționează rețelele neuronale convoluționale?
- Ce este o rețea neuronală?
- Pot folosi Jupyter Notebooks cu Azure Machine Learning?
- Ce este ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Declinarea responsabilității**:
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original, în limba sa nativă, trebuie considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de oameni. Nu ne asumăm răspunderea pentru eventuale neînțelegeri sau interpretări greșite care pot apărea în urma utilizării acestei traduceri.
